In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#import all data
feature_data = pd.read_csv('kaggle_data/features.txt', header=None, sep="  ", names=['feature_names', 'feature_description'], engine='python')
list_feature_names = list(feature_data['feature_names'])
train_data = pd.read_csv('kaggle_data/train.csv', header=None, sep=" ", names=list_feature_names)
test_data=pd.read_csv('kaggle_data/test-val.csv',header=None,sep=" ",names=list_feature_names)
target_data = pd.read_csv('kaggle_data/train-targets.csv', sep=",")

In [3]:
#Data engineering 

# 'weekday' encoding
weekday_data = pd.get_dummies(train_data['weekday'],prefix='weekday',drop_first=True)

# 'category' encoding
category_data = pd.get_dummies(train_data['category'],prefix='category',drop_first=True)

#fusion that shit
other_data = train_data.drop(['weekday','category'],axis=1)
training_data = pd.concat([category_data,weekday_data,other_data],axis=1)

In [6]:
#lets do the same with the test data
weekday_data_test=pd.get_dummies(test_data['weekday'],prefix='weekday',drop_first=True)
category_data_test=pd.get_dummies(test_data['category'],prefix='category',drop_first=True)
other_data_test=test_data.drop(['weekday','category'],axis=1)
testing_data=pd.concat([category_data_test,weekday_data_test,other_data_test],axis=1)

#see it
#testing_data.head(5)


In [16]:
#Data standardization
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(training_data)
X_reg = scaler.transform(training_data)
y_reg = target_data['Prediction'].values

#scale that test data
X_test = scaler.transform(testing_data)

#visualize X_reg
#print X_reg

In [17]:
# Making the kfold
from sklearn import model_selection
kf = model_selection.KFold(n_splits = 5)
k_folds = kf.split(X_reg, y_reg)

In [18]:
# Setting up the Linear regression
from sklearn import linear_model
lin_reg = linear_model.LinearRegression()

In [19]:
# Predictions by Cross validation
y_pred = model_selection.cross_val_predict(lin_reg, X_reg, y_reg, cv=k_folds)

In [21]:
# Let's see the error
from sklearn import metrics
print("Mean squared error: %.3f" % metrics.mean_squared_error(y_reg, y_pred))

Mean squared error: 60730229.243


In [22]:
#test juste fit et val
lin_reg_2 = linear_model.LinearRegression()
lin_reg_2.fit(X_reg, y_reg)
y_pred_2 = lin_reg_2.predict(X_reg)
print("Mean squared error: %.3f" % metrics.mean_squared_error(y_reg, y_pred_2))

Mean squared error: 58406785.185
